# MP2 梯度

在这一节中，我们会回顾 MP2 的梯度性质计算。这一节不涉及 DFT 的计算。

这一节与以后与 MP2 有关的推导，我们参考 C. M. Aikens TCA 2003 (doi: 10.1007/s00214-003-0453-3)。

In [1]:
import numpy as np
from pyscf import scf, gto, lib, grad, hessian, dft, mp
import pyscf.hessian.rks
import pyscf.grad.rks
from functools import partial

from utilities import val_from_fchk, NumericDiff
from hessian import HFHelper, GGAHelper

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.einsum_path = partial(np.einsum_path, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(6, linewidth=150, suppress=True)

In [2]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)

In [3]:
hfh = HFHelper(mol)

## MP2 梯度的高级实现

MP2 梯度可以由 PySCF 直接给出：

In [4]:
mp2_eng = mp.MP2(hfh.scf_eng)
mp2_eng.kernel()[0]

-0.27971024503816694

In [5]:
mp2_grad = grad.mp2.Gradients(mp2_eng)
mp2_grad.kernel()

array([[-0.102293,  0.014371,  0.031588],
       [ 0.008573,  0.75439 , -0.009366],
       [ 0.087807,  0.00276 ,  0.014487],
       [ 0.005914, -0.77152 , -0.036708]])

我们也可以将上述数值与 [Gaussian 结果](include/mp2_grad/mp2_grad.gjf) 进行比较：

In [6]:
np.allclose(
    mp2_grad.de.ravel(),
    val_from_fchk("Cartesian Gradient", "include/mp2_grad/mp2_grad.fchk"),
    atol=1e-8
)

True

## MP2 能量回顾

我们先简单回顾 MP2 能量的实现。注意到我们现在只关心 Restricted 参考态，因此其不管是 MP2 能量还是其导出的梯度，其推导过程与程序实现与 Unrestricted 方法会完全不同。

MP2 能量可以通过下式导出：

$$
E_\mathrm{elec}^\mathrm{MP2} = \frac{(ia|jb) \big( 2 (ia|jb) - (ib|ja) \big)}{D_{ij}^{ab}}
$$

其中，`eri0_mo_iajb` 代表 $(ia|jb)$：

In [7]:
eri0_mo_iajb = hfh.eri0_mo[so, sv, so, sv]

/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:229: UserWarning: eri0_mo: ERI AO -> MO is quite expensive!
  warnings.warn("eri0_mo: ERI AO -> MO is quite expensive!")
/share/home/zyzhu/Git-Repo/Python-xDH/pyxdh/hessian/hf_helper.py:222: UserWarning: eri0_ao: ERI should not be stored in memory! Consider J/K engines!
  warnings.warn("eri0_ao: ERI should not be stored in memory! Consider J/K engines!")


而 `D_iajb` 代表 $D_{ij}^{ab} = \varepsilon_i - \varepsilon_a + \varepsilon_j - \varepsilon_b$：

In [8]:
D_pqrs = lib.direct_sum("i - a + j - b", hfh.e, hfh.e, hfh.e, hfh.e)
D_iajb = D_pqrs[so, sv, so, sv]

这种记号尽管一般不会引起歧义，但仍然需要将其与密度记号 $D_{\mu \nu}$ 区分开。

通过定义上述两个变量，我们可以很轻松地计算 MP2 相关能。

In [9]:
(eri0_mo_iajb * (2 * eri0_mo_iajb - eri0_mo_iajb.swapaxes(1, 3)) * (1 / D_iajb)).sum()

-0.2797102450381638

## MP2 梯度：简易做法

我们下面简单地叙述一种快速但不安全的 MP2 梯度解法。这种不安全性与我们以前推导 Hartree-Fock Hessian 时所产生的 $U_{ij}^{A_t}$ 与 $U_{ab}^{A_t}$ 是一致的；但在 MP2 梯度的处理中，解决这种不安全性的困难所涉及的推导会比较复杂，我们在这份笔记中不会详细叙述。

我们首先为了程序书写方便，定义以下张量：

* `tmp_iajb`: $4 (ia|jb) - 2 (ib|ja)$

* `eri1_mo_iajb`: $(ia|jb)^{A_t}$

* `eri0_mo`: $(pq|rs)$

* `U_1`: $U_{pq}^{A_t}$

* `U_1_vo`: $U_{ai}^{A_t}$

In [10]:
%%capture
tmp_iajb = (4 * eri0_mo_iajb - 2 * eri0_mo_iajb.swapaxes(1, 3))
eri1_mo_iajb = hfh.eri1_mo[:, :, so, sv, so, sv]
eri0_mo = hfh.eri0_mo
U_1 = hfh.U_1
U_1_vo = hfh.U_1_vo

* `e_1`: $\frac{\partial}{\partial A_t} e_p = B_{pp}^{A_t} + A_{pp, ai} U_{ai}^{A_t}$

* `eo_1`: $\frac{\partial}{\partial A_t} e_i$

* `ev_1`: $\frac{\partial}{\partial A_t} e_a$

In [11]:
e_1 = (hfh.B_1 + hfh.Ax0_Core(sa, sa, sv, so)(U_1_vo)).diagonal(0, -1, -2)
eo_1 = e_1[:, :, so]
ev_1 = e_1[:, :, sv]

我们现在简单推导 MP2 梯度；目标是达到类似于 Aikens (30) 的表达式。

\begin{align}
\frac{\partial}{\partial A_t} E_\mathrm{elec}^\mathrm{MP2} 
&= \frac{1}{D_{ij}^{ab}} \big( 2 (ia|jb) - (ib|ja) \big) \frac{\partial}{\partial A_t} (ia|jb) 
\\&\quad+ \frac{1}{D_{ij}^{ab}} (ia|jb) \frac{\partial}{\partial A_t} \big( 2 (ia|jb) - (ib|ja) \big) 
\\&\quad- \big( 2 (ia|jb) - (ib|ja) \big) (ia|jb) \frac{1}{(D_{ij}^{ab})^2} \frac{\partial}{\partial A_t} D_{ij}^{ab}
\\&= \big( 4 (ia|jb) - 2 (ib|ja) \big) \frac{\partial}{\partial A_t} (ia|jb) 
\\&\quad- \big( 4 (ia|jb) - 2 (ib|ja) \big) (ia|jb) \frac{\frac{\partial}{\partial A_t} \varepsilon_i - \frac{\partial}{\partial A_t} \varepsilon_a}{(D_{ij}^{ab})^2}
\\&= \big( 4 (ia|jb) - 2 (ib|ja) \big) (ia|jb)^{A_t}
\\&\quad+ 2 \big( 4 (ia|jb) - 2 (ib|ja) \big) (pa|jb) U_{pa}^{A_t}
\\&\quad+ 2 \big( 4 (ia|jb) - 2 (ib|ja) \big) (ip|jb) U_{pi}^{A_t}
\\&\quad- \big( 4 (ia|jb) - 2 (ib|ja) \big) (ia|jb) \frac{\frac{\partial}{\partial A_t} \varepsilon_i - \frac{\partial}{\partial A_t} \varepsilon_a}{(D_{ij}^{ab})^2}
\end{align}

上述推导中使用了一些 $i, j$ 与 $a, b$ 的对称性。对上式的程序化如下：

In [12]:
E_MP2c_1 = (
    np.einsum("iajb, Atiajb, iajb -> At", tmp_iajb, eri1_mo_iajb, 1 / D_iajb)
    + 2 * np.einsum("iajb, pajb, Atpi, iajb -> At", tmp_iajb, eri0_mo[:, sv, so, sv], U_1[:, :, :, so], 1 / D_iajb)
    + 2 * np.einsum("iajb, ipjb, Atpa, iajb -> At", tmp_iajb, eri0_mo[so, :, so, sv], U_1[:, :, :, sv], 1 / D_iajb)
    - np.einsum("iajb, iajb, iajb, Atia -> At", tmp_iajb, eri0_mo_iajb, 1 / (D_iajb ** 2), lib.direct_sum("Ati - Ata -> Atia", eo_1, ev_1))
)
E_MP2c_1

array([[ 0.037398, -0.002894,  0.05093 ],
       [-0.002857,  0.032367, -0.054274],
       [-0.033579, -0.00045 , -0.00382 ],
       [-0.000963, -0.029024,  0.007164]])

我们可以拿 PySCF 所生成的 MP2 梯度与 HF 梯度的差，来验证上述的相关能梯度是否正确：

In [13]:
mp2_grad.kernel() - hfh.scf_grad.kernel()

array([[ 0.037399, -0.002894,  0.05093 ],
       [-0.002857,  0.032367, -0.054274],
       [-0.033579, -0.00045 , -0.00382 ],
       [-0.000963, -0.029024,  0.007164]])

但显然地，我们不应当满足于此。在下一小节中，我们会通过正常的手段，绕开 U 矩阵中的占据-占据与非占-非占部分，并使用 Z-Vector 方法，实现 MP2 梯度。但推导过程不在这里列出；详细推导过程见 Aikens。但我们会在以 GGA 为参考态的梯度，或 XYG3 梯度推导中，会重新回顾一部分推导过程。

## MP2 梯度：常规做法

下面我们叙述 MP2 的常规做法。常规上，MP2 梯度分为三部分考虑　(Aikens, 24)：

\begin{align}
E_\mathrm{elec}^{\mathrm{MP2}, A_t} &= D_{\mu \nu}^\mathrm{MP2} h_{\mu \nu}^{A_t} + W_{\mu \nu}^\mathrm{MP2} S_{\mu \nu}^{A_t} + \Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2} (\mu \nu | \kappa \lambda)^{A_t}
\\&= D_{pq}^\mathrm{MP2} h_{pq}^{A_t} + W_{pq}^\mathrm{MP2} S_{pq}^{A_t} + \Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2} (\mu \nu | \kappa \lambda)^{A_t}
\end{align}

后文中，我们分别称三部分为弛豫密度贡献、加权密度贡献、以及双粒子密度贡献。

尽管我们说这是常规做法，但仍然使用了一些不合理的算法，其不合理性在于内存占用量。在真正实践 MP2 梯度时，我们还需要更强大的算法。作为文档说明，这里就不考虑内存占用问题。

### 记号说明

下述项可以从 HF 计算后立即获得：

* `e`, `eo`, `ev`: 轨道能量

* `C`, `Co`, `Cv`: 轨道系数

* `D`: RHF 密度 $D_{\mu \nu} = 2 C_{\mu i} C_{\nu i}$

* `eri0_mo`: $(pq|rs)$

* `eri1_ao`: $(\mu \nu | \kappa \lambda)^{A_t}$

* `H_1_mo`: $h_{pq}^{A_t}$

* `S_1_mo`: $S_{pq}^{A_t}$

* `Ax0_Core`: 用于计算 $A_{pq, rs} X_{rs}$ 的函数

下述项可以用于计算 MP2 能量；在计算 MP2 梯度时也经常需要使用：

* `D_iajb`: 如同之前定义过的，$D_{ij}^{ab} = \varepsilon_i - \varepsilon_a + \varepsilon_j - \varepsilon_b$

* `t_iajb`: $t_{ij}^{ab} = (ia|jb) (D_{ij}^{ab})^{-1}$

* `T_iajb`: $T_{ij}^{ab} = 2 t_{ij}^{ab} - t_{ij}^{ba}$

In [14]:
%%capture

e, eo, ev = hfh.e, hfh.eo, hfh.ev
C, Co, Cv = hfh.C, hfh.Co, hfh.Cv
D = hfh.D
eri0_mo = hfh.eri0_mo
eri1_ao = hfh.eri1_ao

H_1_mo = hfh.H_1_mo
S_1_mo = hfh.S_1_mo
Ax0_Core = hfh.Ax0_Core

D_iajb = lib.direct_sum("i - a + j - b", hfh.eo, hfh.ev, hfh.eo, hfh.ev)
t_iajb = eri0_mo[so, sv, so, sv] / D_iajb
T_iajb = 2 * t_iajb - t_iajb.swapaxes(1, 3)

我们可以拿 MP2 能量来简单验证上面定义的最后三个变量是否正确：

$$
E_\mathrm{elec}^\mathrm{MP2} = t_{ij}^{ab} T_{ij}^{ab} D_{ij}^{ab}
$$

In [15]:
np.allclose((t_iajb * T_iajb * D_iajb).sum(), mp2_eng.e_corr)

True

在这里我们顺便说明以后需要定义的变量。MP2 梯度的三部分中，

* 第一部分：弛豫密度 `D_r` $D_{pq}^\mathrm{MP2}$ 与 Lagrangian `L` $L_{ai}^\mathrm{MP2}$

* 第二部分：加权密度 `D_W` $W_{pq}^\mathrm{MP2}$

* 第三部分：双粒子密度 `D_pdm2` $\Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2}$

In [35]:
D_r = None  # (nmo, nmo)
L = None  # (nvir, nocc)
D_W = None  # (nmo, nmo)
D_pdm2 = None  # (nao, nao, nao, nao)

### 弛豫密度 $D_{pq}^\mathrm{MP2}$

MP2 的弛豫密度总的来说还分为两部分：第一部分是比较容易生成的占据-占据与非占-非占部分；第二部分是通过求解 Z-Vector 方程给出的占据-非占与非占-占据部分。

占据-占据与非占-非占部分的贡献是 (Aikens, 177, 178)

\begin{align}
D_{ij}^\mathrm{MP2} &= - 2 T_{ik}^{ab} t_{jk}^{ab} \\
D_{ab}^\mathrm{MP2} &= 2 T_{ij}^{ac} t_{ij}^{bc}
\end{align}

In [36]:
D_r = np.zeros((nmo, nmo))
D_r[so, so] += - 2 * np.einsum("iakb, jakb -> ij", T_iajb, t_iajb)
D_r[sv, sv] += 2 * np.einsum("iajc, ibjc -> ab", T_iajb, t_iajb)

剩下的部分需要通过 Z-Vector 方程构建。Z-Vector 方程的等式左为 (Aikens, 159)

\begin{align}
L_{ai} =& A_{ai, kl} D_{kl}^\mathrm{MP2} + A_{ai, bc} D_{bc}^\mathrm{MP2} \\
&- 4 T_{jk}^{ab} (ij|bk) + 4 T_{ij}^{bc} (ab|jc)
\end{align}

在程序实现中，一般来说我们应当尽量避免多次使用 $A_{pq, rs}$，因此我们可以将上述未生成完全的 `D_r` 变量直接代入 $A_{ai, kl} X_{pq}$ 的计算过程 `Ax0_Core(sv, so, sa, sa)` 中。

In [37]:
L = np.zeros((nvir, nocc))
L += Ax0_Core(sv, so, sa, sa)(D_r)
L -= 4 * np.einsum("jakb, ijbk -> ai", T_iajb, eri0_mo[so, so, sv, so])
L += 4 * np.einsum("ibjc, abjc -> ai", T_iajb, eri0_mo[sv, sv, so, sv])

随后求解 Z-Vector (Aikens, 163) 方程即可。

$$
(\varepsilon_i - \varepsilon_a) D_{ai}^\mathrm{MP2} - A_{ai, bj} D_{bj}^\mathrm{MP2} = L_{ai}
$$

<div class="alert alert-info">

**提示**

在 Aikens 文章中，有几处与矩阵分割有关处需要留意。下面的 $D_{ai}^\mathrm{MP2}$ 是其中一例。需要注意到，$D_{ia}^\mathrm{MP2} = 0$。同时，从现在开始，许多我们接触到的矩阵将不再是对称的了。可以验证，一般地，$D_{ij}^\mathrm{MP2} \neq D_{ji}^\mathrm{MP2}$。

相同的情况还会出现在 $W_{pq}^\mathrm{MP2}$ 的生成过程中。

</div>

In [38]:
D_r[sv, so] = scf.cphf.solve(Ax0_Core(sv, so, sv, so), e, hfh.mo_occ, L, max_cycle=100, tol=1e-13)[0]

就此，我们完整地生成了弛豫密度 `D_r` $D_{pq}^\mathrm{MP2}$ 了。我们可以在这里生成弛豫密度与 Hamiltonian Core Skeleton 导数对 MP2 相关能梯度的贡献大小：

In [39]:
# h[1].DM - Correct with h1ao-dm1 term in PySCF
# Note that dm1 in PySCF includes HF density rdm1.
(D_r * H_1_mo).sum(axis=(-1, -2))

array([[ 0.284392, -0.044988,  0.139081],
       [-0.120538,  0.201388, -0.32821 ],
       [-0.180212,  0.010875, -0.067976],
       [ 0.016358, -0.167275,  0.257105]])

### 加权密度 $W_{pq}^\mathrm{MP2}$

加权密度分为三部分：

第一部分 (Aikens, 181-183)

\begin{align}
W_{ij}^\mathrm{MP2} [\mathrm{I}] &= - 2 T_{ik}^{ab} (ja|kb) \\
W_{ab}^\mathrm{MP2} [\mathrm{I}] &= - 2 T_{ij}^{ac} (ib|jc) \\
W_{ai}^\mathrm{MP2} [\mathrm{I}] &= - 4 T_{jk}^{ab} (ij|bk) \\
W_{ia}^\mathrm{MP2} [\mathrm{I}] &= 0
\end{align}

In [40]:
# W[I] - Correct with s1-im1 term in PySCF
D_WI = np.zeros((nmo, nmo))
D_WI[so, so] = - 2 * np.einsum("iakb, jakb -> ij", T_iajb, eri0_mo[so, sv, so, sv])
D_WI[sv, sv] = - 2 * np.einsum("iajc, ibjc -> ab", T_iajb, eri0_mo[so, sv, so, sv])
D_WI[sv, so] = - 4 * np.einsum("jakb, ijbk -> ai", T_iajb, eri0_mo[so, so, sv, so])

第二部分 (Aikens, 184-186)

\begin{align}
W_{ij}^\mathrm{MP2} [\mathrm{II}] &= - \frac{1}{2} D_{ij}^\mathrm{MP2} (\varepsilon_i + \varepsilon_j) \\
W_{ab}^\mathrm{MP2} [\mathrm{II}] &= - \frac{1}{2} D_{ab}^\mathrm{MP2} (\varepsilon_a + \varepsilon_b) \\
W_{ai}^\mathrm{MP2} [\mathrm{II}] &= - D_{ai}^\mathrm{MP2} \varepsilon_i \\
W_{ia}^\mathrm{MP2} [\mathrm{II}] &= 0
\end{align}

In [41]:
# W[II] - Correct with s1-zeta term in PySCF
# Note that zeta in PySCF includes HF energy weighted density rdm1e
# The need of scaler 1 in D_WII[sv, so] is that Aikens use doubled P
D_WII = np.zeros((nmo, nmo))
D_WII[so, so] = - 0.5 * D_r[so, so] * lib.direct_sum("i + j -> ij", eo, eo)
D_WII[sv, sv] = - 0.5 * D_r[sv, sv] * lib.direct_sum("a + b -> ab", ev, ev)
D_WII[sv, so] = - D_r[sv, so] * eo

第三部分 (Aikens, 187)

$$
W_{ij}^\mathrm{MP2} [\mathrm{III}] = - \frac{1}{2} A_{ij, pq} D_{pq}^\mathrm{MP2}
$$

In [42]:
# W[III] - Correct with s1-vhf_s1occ term in PySCF
D_WIII = np.zeros((nmo, nmo))
D_WIII[so, so] = - 0.5 * Ax0_Core(so, so, sa, sa)(D_r)

我们将这些项相加，就得到了加权密度的矩阵 `D_W` $W_{pq}^\mathrm{MP2}$ 了：

In [43]:
# Summation
D_W = D_WI + D_WII + D_WIII

我们可以在这里生成加权密度与重叠积分 Skeleton 导数对 MP2 相关能梯度的贡献大小：

In [44]:
(D_W * S_1_mo).sum(axis=(-1, -2))

array([[-0.022622,  0.003919,  0.008189],
       [ 0.00357 , -0.0051  , -0.001727],
       [ 0.019548, -0.000737,  0.004041],
       [-0.000496,  0.001918, -0.010503]])

### 双粒子密度 $\Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2}$

双粒子密度可以分为两部分：

第一部分：不可拆分项 (Aikens, 189)，意味着这部分贡献至少需要经过一次 $O(N^5)$ 的积分转换才能得到梯度贡献的部分

$$
\Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2, NS} = T_{ia}^{jb} C_{\mu i} C_{\nu a} C_{\kappa j} C_{\lambda b}
$$

In [45]:
# Non-seperatable - Correct with `de` generated in PySCF code part --2e AO integrals dot 2pdm--
D_pdm2_NS = 2 * np.einsum("iajb, ui, va, kj, lb -> uvkl", T_iajb, Co, Cv, Co, Cv)

第二部分：可拆分项 (Aikens, 190)，意味着可以通过 $O(N^4)$ 的库伦或交换积分过程得到梯度贡献的部分

$$
\Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2, NS} = D_{pq}^\mathrm{MP2} C_{\mu p} C_{\nu q} D_{\kappa \lambda} - \frac{1}{2} D_{pq}^\mathrm{MP2} C_{\mu p} C_{\kappa q} D_{\nu \lambda}
$$

In [46]:
# Seperatable - Correct with vhf1-dm1p term in PySCF
# Note that dm1p in PySCF includes HF density rdm1
D_pdm2_S = np.einsum("uv, kl -> uvkl", C @ D_r @ C.T, D)
D_pdm2_S -= 0.5 * np.einsum("uk, vl -> uvkl", C @ D_r @ C.T, D)

我们将这些项相加，就得到了双粒子密度 `D_pdm2` $\Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2}$ 了：

In [47]:
# Summation
D_pdm2 = D_pdm2_NS + D_pdm2_S

我们可以在这里生成双粒子密度与 ERI 积分 Skeleton 导数对 MP2 相关能梯度的贡献大小：

In [48]:
(D_pdm2 * eri1_ao).sum(axis=(-1, -2, -3, -4))

array([[-0.224372,  0.038176, -0.09634 ],
       [ 0.114112, -0.163921,  0.275663],
       [ 0.127085, -0.010588,  0.060116],
       [-0.016825,  0.136334, -0.239439]])

### 总 MP2 相关能梯度 $E_\mathrm{elec}^{\mathrm{MP2}, A_t}$

最后，我们验证 $E_\mathrm{elec}^{\mathrm{MP2}, A_t}$ 与之前用 PySCF 模块生成的 MP2 相关能梯度是否一致：

In [49]:
np.allclose(
    + (D_r * H_1_mo).sum(axis=(-1, -2))
    + (D_W * S_1_mo).sum(axis=(-1, -2))
    + (D_pdm2 * eri1_ao).sum(axis=(-1, -2, -3, -4)),
    mp2_grad.kernel() - hfh.scf_grad.kernel()
)

True

## 临时文档：XYG3 梯度——简易做法

In [14]:
from hessian import NCGGAEngine

In [15]:
def mol_to_grids(mol):
    grids = dft.gen_grid.Grids(mol)
    grids.atom_grid = (75, 302)
    grids.becke_scheme = dft.gen_grid.stratmann
    grids.prune = None
    grids.build()
    return grids

In [16]:
scfh = GGAHelper(mol, "b3lypg", mol_to_grids(mol))
nch = GGAHelper(mol, "0.8033*HF - 0.0140*LDA + 0.2107*B88, 0.6789*LYP", mol_to_grids(mol), init_scf=False)
ncgga = NCGGAEngine(scfh, nch)

In [17]:
ncgga.E_1

array([[-0.13754,  0.01651, -0.01859],
       [ 0.0091 ,  0.72248,  0.04486],
       [ 0.12171,  0.00314,  0.01583],
       [ 0.00673, -0.74212, -0.04209]])

In [18]:
%%capture
eri0_mo_iajb = scfh.eri0_mo[so, sv, so, sv]
D_pqrs = lib.direct_sum("i - a + j - b", scfh.e, scfh.e, scfh.e, scfh.e)
D_iajb = D_pqrs[so, sv, so, sv]
tmp_iajb = (4 * eri0_mo_iajb - 2 * eri0_mo_iajb.swapaxes(1, 3))
eri1_mo_iajb = scfh.eri1_mo[:, :, so, sv, so, sv]
eri0_mo = scfh.eri0_mo
U_1 = scfh.U_1
U_1_vo = scfh.U_1_vo
e_1 = (scfh.B_1 + scfh.Ax0_Core(sa, sa, sv, so)(U_1_vo)).diagonal(0, -1, -2)
eo_1 = e_1[:, :, so]
ev_1 = e_1[:, :, sv]

In [19]:
E_MP2c_1 = (
    np.einsum("iajb, Atiajb, iajb -> At", tmp_iajb, eri1_mo_iajb, 1 / D_iajb)
    + 2 * np.einsum("iajb, pajb, Atpi, iajb -> At", tmp_iajb, eri0_mo[:, sv, so, sv], U_1[:, :, :, so], 1 / D_iajb)
    + 2 * np.einsum("iajb, ipjb, Atpa, iajb -> At", tmp_iajb, eri0_mo[so, :, so, sv], U_1[:, :, :, sv], 1 / D_iajb)
    - np.einsum("iajb, iajb, iajb, Atia -> At", tmp_iajb, eri0_mo_iajb, 1 / (D_iajb ** 2), lib.direct_sum("Ati - Ata -> Atia", eo_1, ev_1))
)
E_MP2c_1

array([[ 0.10983, -0.00711,  0.13068],
       [-0.00161,  0.05621, -0.1443 ],
       [-0.1057 , -0.00117, -0.00413],
       [-0.00251, -0.04793,  0.01774]])

In [20]:
ncgga.E_1 + 0.3211 * E_MP2c_1

array([[-0.10227,  0.01422,  0.02337],
       [ 0.00859,  0.74053, -0.00148],
       [ 0.08777,  0.00276,  0.0145 ],
       [ 0.00592, -0.75751, -0.0364 ]])

In [21]:
np.allclose(
    (ncgga.E_1 + 0.3211 * E_MP2c_1).ravel(),
    val_from_fchk("Cartesian Gradient", "include/mp2_grad/xyg3_grad.fchk"),
    atol=1e-6, rtol=1e-4
)

True